In [1]:
include("functions.jl")

update_draft (generic function with 1 method)

In [ ]:
#TODO: August 24 ADP data
#TODO: figure out how to rate the players that are never "starters"

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [3]:
function dynasty_sf_model(team, pv, drafted, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # num teams
    P = size(data)[1] # num_players
    Y = 10 # num_years projecting
    R = 26 # roster size

    alpha = 0.5 # starter weight (bench = 1, starter = 1+alpha)
    pv = 0.9 # present value of future years 
    #(0.9 = next year projections discounted by 0.9, two year out projections discounted by 0.9^2 = 0.81, etc.)
    
    min_qbs = 3
    max_qbs = 5
    min_rbs = 7
    max_rbs = 11
    min_wrs = 7
    max_wrs = 11
    min_tes = 2
    max_tes = 4

    # this year and next year as of now
    starting_qbs_now = 2
    starting_rbs_now = 3
    starting_wrs_now = 3
    starting_flex_now = 8
    starting_tes_now = 1

    #years 3-5
    starting_qbs_mid = 2
    starting_rbs_mid = 2
    starting_wrs_mid = 2
    starting_flex_mid = 6
    starting_tes_mid = 1

    #years 6-7
    starting_qbs_fut = 1
    starting_flex_fut = 4
    starting_tes_fut = 1


    opps = players_drafted_opps(team, T)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])

    picks = [team, (2*T+1)-team]
    for i in 3:R
        push!(picks, picks[i-2]+(2*T))
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team
    @variable(model, y[i = 1:P, j = 1:Y], Bin) # whether or not player is "starting" in year j on your roster

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, j+5]*(x[i]+alpha*y[i,j])*(pv^(j-1)) for i in 1:P, j in 1:Y))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # selecting players in y only if on roster
    @constraint(model, [i = 1:P, j = 1:Y], y[i,j] <= x[i])

    # roster must contain R players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= R)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)

    # starting constraints
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in flex) == starting_flex_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in tes) == starting_tes_now)

    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in flex) == starting_flex_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in tes) == starting_tes_mid)

    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in flex) == starting_flex_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in tes) == starting_tes_fut)

    @constraint(model, [j = 8:10], sum(y[i,j] for i in 1:P) >= 2)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    for pick in length(drafted[team])+1:R
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= R+1-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Position = String[], Pick = Int64[], ADP = Float64[])
    count = 1
    for i in drafted[team]
        if i == "QB" || i == "RB" || i == "WR" || i == "TE"
            push!(roster, [i, i, picks[count], 500])
            continue
        end
        push!(roster, [data[player_to_index[i], "Name"], data[player_to_index[i], "Position"], picks[count], data[player_to_index[i], "Dyn_SF"]])
        count += 1
    end
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                push!(roster, [data[i, "Name"], data[i, "Position"], picks[count], data[i, "Dyn_SF"]])
                count += 1
            end
        end
    end
    next_pick = ""
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                next_pick = data[i, "Name"]
                break
            end
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return next_pick, roster
end

dynasty_sf_model (generic function with 2 methods)

In [4]:
data, player_to_index, qbs, rbs, wrs, tes, flex, names = initialize_data()

(491×27 DataFrame
 Row │ FirstName  LastName     Dyn_SF    RedraftHalfPPR  Position  Y0_VORP   Y ⋯
     │ String15   String31     Float64?  Float64?        String3   Float64   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Patrick    Mahomes           1.9            14.4  QB         9.06229  6 ⋯
   2 │ Josh       Allen             2.3            22.6  QB         8.31693  4
   3 │ Jalen      Hurts             3.9            23.7  QB         8.0804   7
   4 │ Joe        Burrow            4.0            35.7  QB         6.92221  6
   5 │ Justin     Jefferson         5.5             1.6  WR        10.5095   8 ⋯
   6 │ Justin     Herbert           6.8            46.3  QB         4.64313  3
   7 │ Ja'Marr    Chase             7.1             3.1  WR         9.97006  6
   8 │ Lamar      Jackson           7.3            34.4  QB         7.18824  5
  ⋮  │     ⋮           ⋮          ⋮            ⋮            ⋮         ⋮        ⋱
 485 │ Tyler      Davi

In [10]:
drafted = update_draft("999801974084202496")

Dict{Any, Any} with 10 entries:
  5  => Any[]
  4  => Any[]
  6  => Any[]
  7  => Any[]
  2  => Any[]
  10 => Any[]
  9  => Any[]
  8  => Any[]
  3  => Any[]
  1  => Any[]

In [6]:
drafted = reset_draft(10)

Dict{Any, Any} with 10 entries:
  5  => Any[]
  4  => Any[]
  6  => Any[]
  7  => Any[]
  2  => Any[]
  10 => Any[]
  9  => Any[]
  8  => Any[]
  3  => Any[]
  1  => Any[]

In [11]:
pv = 0.90
first_choice, roster = dynasty_sf_model(5, pv, drafted);
second_choice, r2 = dynasty_sf_model(5, pv, drafted, [first_choice]);
third_choice, r3 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice])
first_choice, second_choice, third_choice

Total VORP: 399.0


Total VORP: 389.4
Total VORP: 380.5


("Justin Jefferson", "Ja'Marr Chase", "Bijan Robinson")

In [81]:
r2

,Name,Position,Pick,ADP
,String,String,Int64,Float64
1,Justin Jefferson,WR,5,5.5
2,Tyreek Hill,WR,16,23.8
3,Cooper Kupp,WR,25,31.3
4,Josh Jacobs,RB,36,39.1
5,Davante Adams,WR,45,45.6
6,Javonte Williams,RB,56,61.9
7,Joe Mixon,RB,65,72.0
8,Cam Akers,RB,76,84.7
9,Aaron Jones,RB,85,89.4


REDRAFT MODEL

In [56]:
function redraft_model(team, num_teams, drafted, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = num_teams # num teams
    P = size(data)[1] # num_players
    R = 13 # roster size

    min_qbs = 1
    max_qbs = 2
    min_rbs = 4
    max_rbs = 7
    min_wrs = 4
    max_wrs = 7
    min_tes = 1
    max_tes = 2

    starting_qbs_now = 1
    starting_rbs_now = 2
    starting_wrs_now = 2
    starting_flex_now = 7
    starting_tes_now = 1

    alpha = 0.5 # weight on starting players


    opps = players_drafted_opps(team, T)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])

    picks = [team, (2*T+1)-team]
    for i in 3:R
        push!(picks, picks[i-2]+(2*T))
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team
    @variable(model, y[i = 1:P], Bin) # whether or not player is "starting" in year j on your roster

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, "Y0_VORP"]*(x[i]+alpha*y[i]) for i in 1:P))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # selecting players to start only if on roster
    @constraint(model, [i = 1:P], y[i] <= x[i])

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # roster must contain 26 players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= R)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)

    @constraint(model, sum(y[player_to_index[i]] for i in qbs) == starting_qbs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in rbs) >= starting_rbs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in wrs) >= starting_wrs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in flex) == starting_flex_now)
    @constraint(model, sum(y[player_to_index[i]] for i in tes) == starting_tes_now)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    for pick in length(drafted[team])+1:R
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= R+1-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Position = String[], Pick = Int64[], ADP = Float64[])
    count = 1
    for i in drafted[team]
        if i == "QB" || i == "RB" || i == "WR" || i == "TE"
            push!(roster, [i, i, picks[count], 500])
            continue
        end
        push!(roster, [data[player_to_index[i], "Name"], data[player_to_index[i], "Position"], picks[count], data[player_to_index[i], "RedraftHalfPPR"]])
        count += 1
    end
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                push!(roster, [data[i, "Name"], data[i, "Position"], picks[count], data[i, "RedraftHalfPPR"]])
                count += 1
            end
        end
    end
    next_pick = ""
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                next_pick = data[i, "Name"]
                break
            end
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return next_pick, roster
end

redraft_model (generic function with 3 methods)

In [46]:
data, player_to_index, qbs, rbs, wrs, tes, flex, names = initialize_data(false)

(491×27 DataFrame
 Row │ FirstName  LastName     Dyn_SF    RedraftHalfPPR  Position  Y0_VORP   Y ⋯
     │ String15   String31     Float64?  Float64?        String3   Float64   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Justin     Jefferson         5.5             1.6  WR        10.5095   8 ⋯
   2 │ Christian  McCaffrey        13.0             2.8  RB         7.90528  6
   3 │ Ja'Marr    Chase             7.1             3.1  WR         9.97006  6
   4 │ Austin     Ekeler           28.5             4.7  RB         8.14655  5
   5 │ Travis     Kelce            20.3             5.2  TE         7.85383  6 ⋯
   6 │ Cooper     Kupp             31.3             6.1  WR        10.5522   7
   7 │ Tyreek     Hill             23.8             7.6  WR         8.65459  6
   8 │ Bijan      Robinson         11.3             8.0  RB         6.99868  8
  ⋮  │     ⋮           ⋮          ⋮            ⋮            ⋮         ⋮        ⋱
 485 │ Tyler      Davi

In [57]:
drafted = reset_draft(12)

Dict{Any, Any} with 12 entries:
  5  => Any[]
  12 => Any[]
  8  => Any[]
  1  => Any[]
  6  => Any[]
  11 => Any[]
  9  => Any[]
  3  => Any[]
  7  => Any[]
  4  => Any[]
  2  => Any[]
  10 => Any[]

In [58]:
first_choice_r, roster_r = redraft_model(10, 12, drafted);
second_choice_r, r2_r = redraft_model(10, 12, drafted, [first_choice_r]);
third_choice_r, r3_r = redraft_model(10, 12, drafted, [first_choice_r, second_choice_r])
first_choice_r, second_choice_r, third_choice_r

Total VORP: 74.6


Total VORP: 74.6
Total VORP: 74.4


("Patrick Mahomes", "CeeDee Lamb", "Stefon Diggs")

In [60]:
r2_r

,Name,Position,Pick,ADP
,String,String,Int64,Float64
1,CeeDee Lamb,WR,10,13.2
2,Davante Adams,WR,15,15.6
3,Joe Mixon,RB,34,38.2
4,Aaron Jones,RB,39,41.4
5,James Conner,RB,58,62.3
6,Alvin Kamara,RB,63,64.0
7,Deshaun Watson,QB,82,86.5
8,Michael Thomas,WR,87,100.7
9,Kirk Cousins,QB,106,108.3
